In [2]:
import pandas as pd
import os
import spacy
# Install: pip install spacy && python -m spacy download en
#import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load('en')



#print os.getcwd()
f_company = "data/contract_data/fortune1000.xls"
parsed = pd.read_excel(f_company, skiprows=5, dtype = str)

In [3]:
#print parsed['Company']
company_names = parsed['Company']
company_streets = parsed['Address']
company_cities = parsed['City']
company_states = parsed['State']
company_zips = parsed['Zipcode']

In [4]:
N_company = len(company_names)

In [5]:
print N_company

996


In [110]:


contract1 =  "THIS MASTER SERVICE AGREEMENT ( Agreement ) is made and entered into EFFECTIVE_DATE \
( Effective Date ) between CONTRACTOR_NAME ( Contractor ), CONTRACTOR_ADDR, and CONTRACTEE_NAME ( Operator ).  \
Contractor and Operator are also referred to as  party  and collectively as the  parties .  \
Subject to and in consideration of the mutual promises, conditions, and agreements contained herein, \
the receipt and sufficiency of which are hereby acknowledged, the parties agree as follows: \
Kunling LLC has owed Darshi Inc. 20000 dollars on August 1st, 2017."

contract2 =  "THIS MASTER SERVICE AGREEMENT ( Agreement ) is made and entered into EFFECTIVE_DATE \
( Effective Date ) between CONTRACTOR_NAME, CONTRACTOR_ADDR, ( Contractor ) and CONTRACTEE_NAME ( Operator ).  \
Contractor and Operator are also referred to as  party  and collectively as the  parties .  \
Subject to and in consideration of the mutual promises, conditions, and agreements contained herein, \
the receipt and sufficiency of which are hereby acknowledged."

contract3 = "THIS AGREEMENT made effective this EFFECTIVE_DATE by and between CONTRACTEE_NAME hereinafter called \
\"Company\" and CONTRACTOR_NAME, CONTRACTOR_ADDR, hereinafter called \"Contractor,\" whereby, in \
consideration of the covenants and provisions hereinafter provided, this Agreement shall control \
and govern all work performed or \
to be performed by Contractor for Company under verbal or written work orders, purchase orders, \
delivery tickets, voices or other verbal or written agreements between the Parties, relating to \
work to be done by Contractor for Company, and upersede any \
such previous agreements from date hereof until specifically cancelled in writing by either Party, \
anything to the co in any such agreement notwithstanding. This Agreement does not obligate Company \
to order work from Contractor, nor obligate Contractor to accept orders for work"



contract4 = "This Contract is entered into as of the day of EFFECTIVE_DATE: (Effective Date) by and between \
The CONTRACTOR_NAME, a Virginia corporation (Consultant), and the CONTRACTEE_NAME, a municipal corporation and \
home rule unit of local government existing under the Constitution of the State of Illinois, acting through its \
Office of Emergency Management and Communications (City), at Chicago, Illinois."

contract5= "This Master Task Order Contract is entered into as of the EFFECTIVE_DATE \
(Effective Date) by and between CONTRACTOR_NAME, Inc., (Consultant), CONTRACTOR_ADDR, and the CONTRACTEE_NAME, CONTRACTEE_ADDR\
 and home rule unit of local government existing under the Constitution of \
the State of Illinois, acting through its Chicago Department of Public Health (City), at Chicago, Illinois."


    
contract6 =  "THIS MASTER SERVICE AGREEMENT ( Agreement ) is made and entered into EFFECTIVE_DATE \
( Effective Date ) between CONTRACTOR_NAME ( Contractor ), CONTRACTOR_ADDR, and \
CONTRACTEE_NAME ( Operator ), CONTRACTEE_ADDR  \
Contractor and Operator are also referred to as  party  and collectively as the  parties . \
Subject to and in consideration of the mutual promises, conditions, and agreements contained herein, \
the receipt and sufficiency of which are hereby acknowledged, the parties agree as follows: \
Kunling LLC has owed Darshi Inc. 20000 dollars on August 1st, 2017."


contract7 = "This Services Agreement (the Agreement) sets forth terms under that The CONTRACTOR_NAME (Company), CONTRACTOR_ADDR \
shall provide services to \
the CONTRACTEE_NAME (the Client). This Agreement is effective as \
of EFFECTIVE_DATE (Effective Date)."


contract8 = "THIS AGREEMENT (Agreement), which comprises the full and complete agreement of \
the Parties hereto and supersedes all previous agreements between the Parties relating to the \
subject matter hereof, is entered into the EFFECTIVE_DATE, \
by and between CONTRACTEE_NAME, CONTRACTEE_ADDR, \
for itself and its wholly owned subsidiaries (Company), \
and CONTRACTOR_NAME, CONTRACTOR_ADDR, (Contractor). \
Company and Contractor are sometimes referred to hereinafter individually as a \
Party or collectively as the Parties."


contract9 = "This contract serves as the Master Services Agreement \
(Agreement) under which orders may be issued for professional services to \
be performed by the division of CONTRACTOR_NAME listed in the \
Addresses and Points of Contact section of this Agreement. This Agreement  \
is entered into by CONTRACTOR_NAME, CONTRACTOR_ADDR, \
and CONTRACTEE_NAME, CONTRACTEE_ADDR,  and is effective as of \
date signed. This agreement is effective from EFFECTIVE_DATE. \
For the purposes of this agreement, CONTRACTOR_NAME and the Customer may be referred to as such or \
as the Parties to this agreement."


contract10 = "This contract is entered into on this EFFECTIVE_DATE, by and between CONTRACTOR_NAME, a \
municipal corporation of the State of California and The CONTRACTEE_NAME, CONTRACTEE_ADDR. \
This contract consists of \
the following documents:"

contract_templates = [contract1, contract2, contract3, contract4, contract5, contract6, contract7, contract8, contract9, contract10]
N_template = len(contract_templates)

In [111]:
from datetime import datetime
import random

def random_day():
    year = random.choice(range(1950, 2018))
    month = random.choice(range(1, 13))
    if month == 2:
        if year % 4 == 0:
            day = random.choice(range(1, 30))
        else:
            day = random.choice(range(1, 29))
    elif month in [1, 3, 5, 7, 8, 10, 12]:
        day = random.choice(range(1, 32))
    else:
        day = random.choice(range(1, 31))
    random_date = datetime(year, month, day)
    return random_date

def random_day_str():
    random_date = random_day()
    
    format_choice = random.choice(["%B %d, %Y","%b %d, %Y", "%m %d, %Y", "%B %d, %Y", "%B %d, %y", "%m %d, %y","%D", "%F"])
    random_date_str = random_date.strftime(format_choice)
    return  random_date_str


In [112]:
def qa_gen(contractor_name, contractee_name, contractor_addr, contractee_addr, effective_date):
    answers = [contractor_name, contractee_name, contractor_addr, contractee_addr, effective_date]
    types = ["contractor name", "contractee name", "contractor address", "contractee address", "effective date"]
    ques_ans_list = []
    for i, ans in enumerate(answers):
        idx_begin = ans[0]
        idx_end = ans[1]
        question = "What is the " + types[i] + "?"
        qa = question + "\t" + str(idx_begin) + " " + str(idx_end)
        ques_ans_list.append(qa)
    return ques_ans_list

In [122]:

from nltk.tokenize import sent_tokenize
import os
import string

if 0:
    def find_entity_index(contract, entity):
            b = contract.find(entity)
            e = b + len(entity)
            idx = (b, e)
            if b == -1:
                return(-1, -1)
            return idx

        
def find_entity_index(sentence, entity):
    sentence_words = nlp.tokenizer(unicode(sentence))
    entity_words = nlp.tokenizer(unicode(entity))
    q = 0
    len_entity = len(entity_words)
    res = (-1, -1)
    while q < len(sentence_words) - len_entity:
        #if entity_words == sentence_words[q:q+len_entity]:
        if 0 :
            print
            print entity_words
            print sentence_words[q:q+len_entity]
            print check_euqal_tokens(entity_words, sentence_words[q:q+len_entity])
            print
        if check_euqal_tokens(entity_words, sentence_words[q:q+len_entity]):
            res = (q, q+len_entity)
            break
        q += 1
        
    if 0:
        print sentence_words
        print entity_words
        print res
        print 
        
        
    return res
    
def check_euqal_tokens(t1, t2):
    if len(t1) != len(t2):
        return False
    if not t1 and not t2:
        return True
    q = 0
    while q < len(t1):
        s1 = str(t1[q])
        s2 = str(t2[q])
        s1 = "".join(l for l in s1 if l not in string.punctuation)
        s2 = "".join(l for l in s2 if l not in string.punctuation)
        if s1 != s2:
            return False
        q += 1
    return True
    
    
    

    
    print sentence
    print entity
    print (start, end)
    return (start , end)

def generate_contract_qa(file_path, N_contracts):
    # file_path = "contracts_qa.txt"
    if os.path.isfile(file_path):
        os.remove(file_path)
    f = open(file_path, "a")

    # N_contracts = 10
    for i in range(N_contracts):
        r_contractor = random.randint(0, N_company-1)
        r_contractee = random.randint(0, N_company-1)
        while r_contractor == r_contractee:
            r_contractee = random.randint(0, N_company-1)
            
        contractor_name = company_names[r_contractor]
        contractor_addr = ", ".join([company_streets[r_contractor], company_cities[r_contractor], \
                                   company_states[r_contractor], company_zips[r_contractor]])
        contractee_name = company_names[r_contractee]
        contractee_addr = ", ".join([company_streets[r_contractee], company_cities[r_contractee], \
                                   company_states[r_contractee], company_zips[r_contractee]])
        effective_date = random_day_str()
        r_template = random.randint(0, N_template-1)
        contract = contract_templates[r_template]
        
        contract = contract.replace("CONTRACTOR_NAME", contractor_name)
              
        contractor_name_idx = find_entity_index(contract, contractor_name)
        #print contractor_name_idx
        
        contract = contract.replace("CONTRACTEE_NAME", contractee_name)
        contractee_name_idx = find_entity_index(contract, contractee_name)
        #print contractee_name_idx
        
        
        contract = contract.replace("CONTRACTOR_ADDR", contractor_addr)
        contractor_addr_idx = find_entity_index(contract,  contractor_addr)
        #print contractor_addr_idx
        
        contract = contract.replace("CONTRACTEE_ADDR", contractee_addr)
        contractee_addr_idx = find_entity_index(contract,  contractee_addr)
        #print contractee_addr_idx
        
        contract = contract.replace("EFFECTIVE_DATE", effective_date)
        effective_date_idx = find_entity_index(contract,   effective_date)
        #print effective_date_idx
        
        contract_sents = sent_tokenize(contract)
  

        ques_ans_list = qa_gen(contractor_name_idx, contractee_name_idx, contractor_addr_idx, contractee_addr_idx, effective_date_idx)

        for i, sent in enumerate(contract_sents):
            content = str(i+1) + " " + sent + "\n"
            f.write(content)
        for k, sent in enumerate(ques_ans_list):
            content = str(i+k+2) + " " + sent  + "\n"
            f.write(content)


    f.close()

    
    
generate_contract_qa("data/contract_data/contracts_train.txt", 1000)
generate_contract_qa("data/contract_data/contracts_test.txt", 1000)

In [40]:
a = [1, 2]
b = [1, 'a']

In [41]:
print a == b

False


In [50]:
EXAMPLE_TEXT = "Hello Mr. Smith, how are you doing today? The weather is great, and Python is awesome. The sky is pinkish-blue. You shouldn't eat cardboard."

In [52]:
from nltk.tokenize import sent_tokenize, word_tokenize
print(word_tokenize(EXAMPLE_TEXT))

['Hello', 'Mr.', 'Smith', ',', 'how', 'are', 'you', 'doing', 'today', '?', 'The', 'weather', 'is', 'great', ',', 'and', 'Python', 'is', 'awesome.', 'The', 'sky', 'is', 'pinkish-blue.', 'You', 'should', "n't", 'eat', 'cardboard', '.']


Hello


In [62]:
for d in doc:
    print d

Hello
Mr.
Smith
,
how
are
you
doing
today
?
The
weather
is
great
,
and
Python
is
awesome
.
The
sky
is
pinkish
-
blue
.
You
should
n't
eat
cardboard
.


In [67]:
print len(doc)

33


In [127]:
doc2 = nlp.tokenizer(unicode("I love you"))
print str(doc2[0]) == str(doc2[0])
print str(doc2[0]).lower()

True
i
